<a href="https://colab.research.google.com/github/sssanghn/AI-Emotion-Diary/blob/main/KoBERT_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT 감정 다중분류모델

> KoBERT를 이용하여 한국어 문장을 여러 클래스로 분류하는 모델을 만들어 보려고 한다.
</br>
공포, 놀람, 분노, 슬픔, 중립, 기쁨, 혐오와 같은 감정이 느껴지는 짧은 대화 텍스트를 각각 어떠한 감정의 텍스트인지 분류하는 모델을 만드는 것이다. </br>
AIHUB에서 가져온 데이터셋으로 KoBERT 모델을 FineTuning하여 </br>
AI 감정일기 어플리케이션에서 작성한 일기 텍스트에서 감정을 추출하기 위한 목적으로 프로젝트를 진행하였다.

**KoBERT란?**

> BERT는 약 33억 개의 단어로 pretrain 되어 있는 기계번역 모델이다. </br>
하지만 외국에서 만든 것이다 보니 영어에 대해서는 정확도는 높고, 한국어에 대해서는 정확도가 떨어진다.</br>
따라서 BERT 모델을 한국어에도 잘 활용할 수 있도록 만들어진 것이 KoBERT이다. </br>
KoBERT는 BERT 모델에서 한국어 데이터를 추가로 학습시킨 모델로, 한국어 위키에서 5백만개의 문장과 5천4백만개의 단어를 학습시킨 모델이다. </br>
따라서 한국어 데이터에 대해서도 높은 정확도를 낼 수 있다고 한다. </br>
한국어의 불규칙한 언어 변화의 특성을 반영하기 위해 데이터 기반 토큰화 (Tokenization) 기법을 적용하여 기존 대비 27%의 토큰만으로 2.6% 이상의 성능 향상을 이끌어 낸 모델이다. </br>

# 1. 프로젝트 설명

**KoBERT 파인튜닝**
> BERT 모델은 자신의 사용 목적에 따라 파인튜닝(FineTuning)이 가능하다. </br>
따라서 Output layer만 추가로 달아주면 원하는 결과를 출력해내는 기계번역 모델을 만들어 낼 수 있다. </br>
이 점을 활용하여 KoBERT 모델을 FineTuning하여 최종적으로 일기 텍스트에서 감정을 추출해내는 모델을 새로 만들어 보려고 한다.

# 2. 데이터 수집

**데이터 수집**
> 이번 프로젝트를 위해 사용한 한국어 대화 문장 데이터셋은 총 3개이다. </br>
</br>
1. 한국어 감정 정보가 포함된 단발성 대화 데이터셋
* SNS 글 및 온라인 댓글에 대한 웹 크롤링을 실시하여 선정된 AIHUB의 공공 데이터셋
* 약 3만 9천건의 데이터 셋 (기쁨, 슬픔, 놀람, 분노, 공포, 혐오, 중립의 7개 감정)
</br>
</br>
2. 한국어 감정 정보가 포함된 연속적 대화 데이터셋
* SNS 글 및 온라인 댓글에 대한 웹 크롤링을 실시하여 선정된 AIHUB의 공공 데이터셋
* 약 1만 건(단발성 55,627문장)의 데이터 셋 (기쁨, 슬픔, 놀람, 분노, 공포, 혐오, 중립의 7개 감정)
</br>
</br>
3. 감성 대화 말뭉치
* 한국어 감성 데이터셋을 상세한 감정 라벨링(감정 대분류와 소분류로 구분)과 함께 제공하는 AIHub의 공공 데이터셋
* 약 4만 건의 데이터 셋 (기쁨, 슬픔, 당황, 분노, 불안, 상처의 6개 감정)
* 데이터 병합을 위해 감정 대분류와 대화 문장을 남기고, 감정 소분류는 누락

# 3. 데이터 전처리

**데이터 전처리**
>

> 한국어 단발성 대화 데이터셋 (7) </br>
: 공포 놀람 분노 슬픔 중립 행복 혐오 </br>
한국어 연속적 대화 데이터셋 (7) </br>
: 공포 놀람 분노 슬픔 중립 행복 혐오 </br>
감성 대화 말뭉치 (6) </br>
: 기쁨 당황 분노 불안 상처 슬픔 </br>
</br>
<병합된 결과> </br>
한국어 단발성 대화 데이터셋 + 한국어 연속적 대화 데이터셋 + 감성 대화 말뭉치 (7) </br>
: 공포 놀람 분노 행복 슬픔   중립 혐오 (7) </br>
: 불안 (----)  분노 기쁨 슬픔,상처 (5) </br>


데이터셋을 살펴본 결과, 감성 대화 말뭉치의 '당황' 감정은 한국어 단발성,연속적 대화 데이터셋의 '공포', '놀람' 감정과 중복된다. </br>
따라서 두 데이터셋을 병합하고, 겹치지 않는 '당황' 감정은 제거하였다.


In [64]:
import pandas as pd

data1 = pd.read_excel('/content/drive/MyDrive/dataset/한국어_단발성_대화_데이터셋.xlsx')

data1.head()

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0


In [ ]:
data2 = pd.read_excel('/content/drive/MyDrive/dataset/한국어_연속적_대화_데이터셋.xlsx')

data2.head()

In [ ]:
data3_train = pd.read_excel('/content/drive/MyDrive/dataset/감성대화말뭉치Training.xlsx')

data3_train.head()

In [ ]:
import pandas as pd

data1 = pd.read_excel('/content/drive/MyDrive/dataset/한국어_단발성_대화_데이터셋.xlsx')
data2 = pd.read_excel('/content/drive/MyDrive/dataset/한국어_연속적_대화_데이터셋.xlsx')
data3_train = pd.read_excel('/content/drive/MyDrive/dataset/감성대화말뭉치Training.xlsx')
data3_validation = pd.read_excel('/content/drive/MyDrive/dataset/감성대화말뭉치Validation.xlsx')

data3 = pd.concat([data3_train, data3_validation])


# 감성 레이블을 정수로 변환 (0~6)
data1.loc[(data1['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
data1.loc[(data1['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
data1.loc[(data1['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
data1.loc[(data1['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
data1.loc[(data1['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
data1.loc[(data1['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
data1.loc[(data1['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

data1_list = []
for q, label in zip(data1['Sentence'], data1['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data1_list.append(data)

# 감성 레이블을 정수로 변환 (0~6)
data2.loc[(data2['Unnamed: 2'] == "공포"), 'Unnamed: 2'] = 0  #공포 => 0
data2.loc[(data2['Unnamed: 2'] == "놀람"), 'Unnamed: 2'] = 1  #놀람 => 1
data2.loc[(data2['Unnamed: 2'] == "분노"), 'Unnamed: 2'] = 2  #분노 => 2
data2.loc[(data2['Unnamed: 2'] == "슬픔"), 'Unnamed: 2'] = 3  #슬픔 => 3
data2.loc[(data2['Unnamed: 2'] == "중립"), 'Unnamed: 2'] = 4  #중립 => 4
data2.loc[(data2['Unnamed: 2'] == "행복"), 'Unnamed: 2'] = 5  #행복 => 5
data2.loc[(data2['Unnamed: 2'] == "혐오"), 'Unnamed: 2'] = 6  #혐오 => 6

# 데이터셋에서 'Unnamed' 열에 결측치가 있으므로 제거
data2.dropna(subset = ['Unnamed: 2'])
data2.drop(0, axis=0, inplace=True)

data2_list = []
for q, label in zip(data2['Unnamed: 1'], data2['Unnamed: 2']) :
    data = []
    data.append(q)
    data.append(str(label))

    data2_list.append(data)

# 감성 레이블을 정수로 변환 (0, 2, 3, 5)
data3.loc[(data3['감정_대분류'] == "불안"), '감정_대분류'] = 0 #불안 => 0
data3.loc[(data3['감정_대분류'] == "분노"), '감정_대분류'] = 2 #분노 => 2
data3.loc[(data3['감정_대분류'] == "슬픔"), '감정_대분류'] = 3 #슬픔 => 3
data3.loc[(data3['감정_대분류'] == "상처"), '감정_대분류'] = 3 #상처 => 3
data3.loc[(data3['감정_대분류'] == "기쁨"), '감정_대분류'] = 5 #기쁨 => 5

# 데이터셋에서 나누어져있는 '사람문장1, 사람문장2, 사람문장3'을 병합, 당황 감정 제거
data3 = data3.drop(data3[data3['감정_대분류'] == '당황'].index)
data3['사람문장'] = data3['사람문장1'].astype(str) + data3['사람문장2'].astype(str) + data3['사람문장3'].astype(str)

data3_list = []
for q, label in zip(data3['사람문장'], data3['감정_대분류'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data3_list.append(data)

merge_data = data1_list + data3_list

# Colab 환경 설정

> 모델을 구현하기에 앞서 코랩 환경설정을 진행하려고 한다. </br>
해당 사양은 KoBERT 오픈소스 내 requirements.txt를 참고하였다.

In [ ]:
# Colab 환경 설정
# requirements : https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/requirements.txt
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

> 다음으로 깃허브 내 파일을 Colab으로 가져온다.

In [ ]:
# https://github.com/SKTBrain/KoBERT의 파일들을 Colab으로 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

> 그 다음으로는 해당 라이브러리들을 import 해준다.

In [ ]:
# KoBERT
from kobert_tokenizer import KoBERTTokenizer

# Transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda:0")

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

# Setting parameters
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):

        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

> BERT 데이터셋을 구성하는 라벨의 개수를 num_classes로 설정할 수 있다.

In [ ]:
# class BERTClassifier
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [ ]:
train_set, test_set = train_test_split(merge_data, test_size=0.2, shuffle=True, random_state=20)

data_train = BERTDataset(train_set, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(test_set, 0, 1, tokenizer, vocab, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=2)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
torch.save(model, f'/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisKOBert.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisKOBert_StateDict.pt')